In [ ]:
!pip install -U -q google-generativeai

In [1]:
import google.generativeai as genai
import json
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown

import pandas as pd

import time
import os

In [9]:
API_KEY = os.environ.get('GEMINI_API_KEY')
genai.configure(api_key=API_KEY)


In [10]:
generation_config = {
  "temperature": 0.8,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 1024,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]


In [11]:
SYSTEM_PROMPT = """Твоя задача - проанализировать текст, который будет отправлен в сообщении и выделить в нем агитационные или мотивационные выражения, лексические конструкции с эмоциональным окрасом и лексические конструкции с политическим окрасом.

Не выделяй исторические факты, такие как даты, события, названия соглашений, договоров, войн или сражений, как политические выражения. В список 'politics' включай только те выражения, которые имеют ярко выраженный негативный политический окрас, содержат признаки национализма, экстремизма, фашизма или призывы к насилию. Например, выражение "Государственная дума" или "Парламент" включать не надо.

Например: "Вперед, за Спартак!", "Великолепный автор", "Самый великий и признанный" -  степени величия, "Гитлер - урод" - уничижение, "Евреев надо убивать", "Хохлы проиграют войну" - выражения с негативным политическим окрасом, с признаками национализма и призыва к насилию.

Формат вывода - dict, без лишних символов до начала dict и после него:
{'prediction': ,'agitation': ,'emotions': ,'politics':}

prediction - предсказание того, является ли текст стилистически нейтральным или нет (1 - если текст нейтральный и 0 - если текст не нейтральный);
agitation - список агитационных или мотивационных выражений, если их нет, то пустой список;
emotions - список лексических конструкций с эмоциональным окрасом, если их нет, то пустой список;
politics - список лексических конструкций с политическим окрасом, если их нет, то пустой список.

В каждом списке не должно быть повторяющихся или очень похожих выражений.
"""

In [12]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=SYSTEM_PROMPT,
                              safety_settings=safety_settings)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
class RequestGemini:
  def __init__(self, gen_cfg: dict,
               safety_set: list,
               sys_instr: str = SYSTEM_PROMPT,
               model: str = "gemini-1.5-pro-latest"):

    self.gemini_model = genai.GenerativeModel(model_name=model,
                              generation_config=gen_cfg,
                              system_instruction=sys_instr,
                              safety_settings=safety_set)

  def process_text_labelling(self, msg: str):

    convo = self.gemini_model.start_chat(history=[])
    convo.send_message(msg)

    return convo.last.text

In [18]:
manager = RequestGemini(generation_config, safety_settings, SYSTEM_PROMPT)

In [19]:
data = pd.read_csv('drive/MyDrive/latest_data.csv')


In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29254 entries, 0 to 29253
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    29240 non-null  object
 1   type    29254 non-null  object
 2   target  29254 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 685.8+ KB


In [21]:
data = data[data.text.isna() == False]

Для ускорения разметки датасета и последующего обучения на нем языковой модели оставим все записи с **target** == 1 и 10000 записей с **target** == 0

In [23]:
target_1 = data[data["target"] == 1]

# Выбираем 10000 строк с target == 0
target_0 = data[data["target"] == 0].sample(n=10000, random_state=42)

In [24]:
balanced_data = pd.concat([target_1, target_0], ignore_index=True)
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [26]:
balanced_data.iloc[1000:][balanced_data]

,text,type,target
1000,марина орлова орлова марина викторовна российс...,"['Статья не проходила экспертную проверку', 'О...",0
1001,золотая серп и молот ссср учреждена указом пре...,"['Статья не проходила экспертную проверку', 'О...",0
1002,тбилиси страны стран участников спортсмен меда...,"['Статья не проходила экспертную проверку', 'О...",0
1003,портрет п а столыпина картина ильи репина напи...,"['Статья не проходила экспертную проверку', 'О...",0
1004,александр иванович смирнов смирнов александр и...,"['Статья не проходила экспертную проверку', 'О...",0
...,...,...,...
13653,праздники и памятные дни национальные украина ...,"['Статья не проходила экспертную проверку', 'О...",0
13654,флага герб герба страна бразилия региона админ...,"['Статья не проходила экспертную проверку', 'О...",0
13655,флага герб герба страна испания региона автоно...,"['Статья не проходила экспертную проверку', 'О...",0
13656,отец мать супруг дети награды и премии викискл...,['Готовая статья'],1


In [27]:
list_of_gemini_preds = []

In [ ]:
import pickle
iteration = 0
for idx, row in balanced_data.iterrows():
    try:
      text = row['text']
      preds = manager.process_text_labelling(msg=text)
      list_of_gemini_preds.append({'text': text, 'gigachat_preds': preds, 'index': idx})

      print(f'Обработан {iteration} текст')
      iteration += 1

      if iteration % 100 == 0:
        with open(f'drive/MyDrive/gemini_preds_{iteration}.pickle', 'wb') as f:
          pickle.dump(list_of_gemini_preds, f)


      time.sleep(20)
    except Exception as e:  # openai.error.RateLimitError:
      print(f"Failed: {e}")
      iteration += 1
      time.sleep(20)
      continue



Обработан 1185 текст
Обработан 1186 текст
Обработан 1187 текст
Обработан 1188 текст
Обработан 1189 текст
Обработан 1190 текст
Обработан 1191 текст
Обработан 1192 текст
Обработан 1193 текст
Обработан 1194 текст
Обработан 1195 текст
Обработан 1196 текст
Обработан 1197 текст
Обработан 1198 текст
Обработан 1199 текст
Обработан 1200 текст
Обработан 1201 текст
Обработан 1202 текст
Обработан 1203 текст
Обработан 1204 текст
Обработан 1205 текст
Обработан 1206 текст
Обработан 1207 текст
Обработан 1208 текст
Обработан 1209 текст
Обработан 1210 текст
Обработан 1211 текст
Обработан 1212 текст
Обработан 1213 текст
Обработан 1214 текст
Обработан 1215 текст


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 16441.90ms


Failed: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Обработан 1217 текст
Обработан 1218 текст
Обработан 1219 текст
Обработан 1220 текст
Обработан 1221 текст
Обработан 1222 текст
Обработан 1223 текст
Обработан 1224 текст
Обработан 1225 текст
Обработан 1226 текст
Обработан 1227 текст
Обработан 1228 текст
Обработан 1229 текст
Обработан 1230 текст
Обработан 1231 текст
Обработан 1232 текст
Обработан 1233 текст
Обработан 1234 текст
Обработан 1235 текст
Обработан 1236 текст
Обработан 1237 текст
Обработан 1238 текст


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 29610.71ms


Failed: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Обработан 1240 текст
Обработан 1241 текст
Обработан 1242 текст
Обработан 1243 текст
Обработан 1244 текст
Обработан 1245 текст
Обработан 1246 текст
Обработан 1247 текст
Обработан 1248 текст
Обработан 1249 текст


...

Failed: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Обработан 1889 текст
Обработан 1890 текст
Обработан 1891 текст
Обработан 1892 текст
Обработан 1893 текст
Обработан 1894 текст
Обработан 1895 текст
Обработан 1896 текст
Обработан 1897 текст
Обработан 1898 текст
Обработан 1899 текст
Обработан 1900 текст
Обработан 1901 текст
Обработан 1902 текст
Обработан 1903 текст
Обработан 1904 текст
Обработан 1905 текст
Обработан 1906 текст
Обработан 1907 текст
Обработан 1908 текст
Обработан 1909 текст


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 8490.61ms


Failed: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Обработан 1911 текст
Обработан 1912 текст
Обработан 1913 текст
Обработан 1914 текст
Обработан 1915 текст
Обработан 1916 текст
Обработан 1917 текст
Обработан 1918 текст
Обработан 1919 текст
Обработан 1920 текст
Обработан 1921 текст
Обработан 1922 текст
Обработан 1923 текст
Обработан 1924 текст
Обработан 1925 текст
Обработан 1926 текст
Обработан 1927 текст
Обработан 1928 текст
Обработан 1929 текст
Обработан 1930 текст
Обработан 1931 текст
Обработан 1932 текст
Обработан 1933 текст
Обработан 1934 текст


ERROR:tornado.access:500 POST /v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 23806.19ms


Failed: 500 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting
Обработан 1936 текст
Обработан 1937 текст
Обработан 1938 текст
Обработан 1939 текст
Обработан 1940 текст
Обработан 1941 текст
Обработан 1942 текст
Обработан 1943 текст
Обработан 1944 текст
Обработан 1945 текст
Обработан 1946 текст
Обработан 1947 текст
Обработан 1948 текст
Обработан 1949 текст
Обработан 1950 текст
Обработан 1951 текст
Обработан 1952 текст
Обработан 1953 текст
Обработан 1954 текст
Обработан 1955 текст
Обработан 1956 текст
Обработан 1957 текст
Обработан 1958 текст
Обработан 1959 текст
Обработан 1960 текст
Обработан 1961 текст
Обработан 1962 текст
Обработан 1963 текст
Обработан 1964 текст
Обработан 1965 текст
Обработан 1966 текст
Обработан 1967 текст
Обработан 1968 текст
Обработан 1969 текст
Обработан 1970 текст
